<a href="https://colab.research.google.com/github/olgagasowska/Machine-Learning-for-Linguists/blob/main/GRU_model_PyTorch_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:

import numpy as np
import torch
import torch.nn as nn

In [ ]:
import torch
import torch.nn as nn

class GRUCell(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, device='cpu'):
        """
        Initialize a GRU cell with Xavier-initialized weights.
        :param input_size: Number of input features.
        :param hidden_size: Number of units in the hidden layer.
        :param device: The device to run the computations on.
        """
        super(GRUCell, self).__init__()
        self.hidden_size = hidden_size
        self.device = device

        fan_in = input_size + hidden_size
        fan_out = hidden_size
        limit = (6 / (fan_in + fan_out)) ** 0.5

        self.Wz = nn.Parameter(torch.empty(input_size + hidden_size, hidden_size, device=device))
        self.Wr = nn.Parameter(torch.empty(input_size + hidden_size, hidden_size, device=device))
        self.Wh = nn.Parameter(torch.empty(input_size + hidden_size, hidden_size, device=device))

        nn.init.uniform_(self.Wz, -limit, limit)
        nn.init.uniform_(self.Wr, -limit, limit)
        nn.init.uniform_(self.Wh, -limit, limit)

        self.bz = nn.Parameter(torch.zeros(hidden_size, device=device))
        self.br = nn.Parameter(torch.zeros(hidden_size, device=device))
        self.bh = nn.Parameter(torch.zeros(hidden_size, device=device))

    def forward(self, input_t: torch.Tensor, h_prev: torch.Tensor) -> torch.Tensor:
        """
        Forward pass for a single GRU cell.
        :param input_t: Input at time step t (batch_size x input_size).
        :param h_prev: Hidden state from the previous time step (batch_size x hidden_size).
        :return: Updated hidden state.
        """
        concat = torch.cat((input_t, h_prev), dim=1)

        z_t = torch.sigmoid(torch.matmul(concat, self.Wz) + self.bz)

        r_t = torch.sigmoid(torch.matmul(concat, self.Wr) + self.br)

        concat_reset = torch.cat((input_t, r_t * h_prev), dim=1)
        h_hat_t = torch.tanh(torch.matmul(concat_reset, self.Wh) + self.bh)

        h_t = (1 - z_t) * h_prev + z_t * h_hat_t

        return h_t



class RecurrentLayer(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, cell_type: str = 'GRU', device='cpu'):
        """
        Initialize a recurrent layer with specified cell type (RNN, GRU, or LSTM).
        :param input_size: Number of input features.
        :param hidden_size: Number of units in the hidden layer.
        :param cell_type: Type of recurrent cell ('RNN', 'GRU', 'LSTM').
        """
        super(RecurrentLayer, self).__init__()
        self.hidden_size = hidden_size
        self.device = device

        if cell_type == 'RNN':
            self.cell = RNNCell(input_size, hidden_size, device=device)
        elif cell_type == 'GRU':
            self.cell = GRUCell(input_size, hidden_size, device=device)
        elif cell_type == 'LSTM':
            self.cell = LSTMCell(input_size, hidden_size, device=device)
        else:
            raise ValueError("Unsupported cell type. Choose from 'RNN', 'GRU', or 'LSTM'.")
        self.cell_type = cell_type

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        """
        Forward pass through the recurrent layer for a sequence of inputs.
        :param inputs: Input sequence (batch size x sequence length x input size).
        :return: Output sequence (batch size x sequence length x hidden size).
        """
        inputs = inputs.to(self.device)
        batch_size, seq_len, _ = inputs.shape

        h_t = torch.zeros(batch_size, self.hidden_size, device=self.device)
        c_t = torch.zeros(batch_size, self.hidden_size, device=self.device) if self.cell_type == 'LSTM' else None

        outputs = []

        for t in range(seq_len):
            input_t = inputs[:, t, :]
            if self.cell_type == 'LSTM':
                h_t, c_t = self.cell(input_t, h_t, c_t)
            else:
                h_t = self.cell(input_t, h_t)
            outputs.append(h_t)

        output_tensor = torch.stack(outputs, dim=1)
        return output_tensor



class SimpleRecurrentNetwork:
    def __init__(self, input_size: int, hidden_size: int, output_size: int, cell_type: str = 'GRU', learning_rate: float = 0.001):
        self.learning_rate = learning_rate
        self.recurrent_layer = RecurrentLayer(input_size, hidden_size, cell_type)
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)
        self.bias_output = np.zeros((1, output_size))
        self.sigmoid = lambda x: 1 / (1 + np.exp(-x))

    def forward(self, inputs: np.ndarray) -> np.ndarray:
        outputs = self.recurrent_layer.forward(inputs)
        final_outputs = []
        for t in range(outputs.shape[1]):
            h_t = outputs[:, t, :]
            output_t = self.sigmoid(np.dot(h_t, self.weights_hidden_output) + self.bias_output)
            final_outputs.append(output_t)
        return np.stack(final_outputs, axis=1)

    def compute_loss(self, predictions: np.ndarray, targets: np.ndarray) -> float:
        predictions_clipped = np.clip(predictions, 1e-10, 1 - 1e-10)
        loss = -np.mean(targets * np.log(predictions_clipped) + (1 - targets) * np.log(1 - predictions_clipped))
        return loss

    def train(self, inputs: np.ndarray, targets: np.ndarray, epochs: int = 1000):
        for epoch in range(epochs):
            predictions = self.forward(inputs)
            loss = self.compute_loss(predictions, targets)
            if epoch % 100 == 0:
                print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")

    def predict(self, inputs: np.ndarray) -> np.ndarray:
        return self.forward(inputs)

inputs = np.random.randn(2, 5, 3)






import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer
from datasets import load_dataset

dataset = load_dataset("stanfordnlp/imdb")

model_id = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)

def process(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(process, batched=True)

def convert_to_tensors(dataset):
    input_ids = torch.tensor(dataset["input_ids"])
    labels = torch.tensor(dataset["label"])
    return TensorDataset(input_ids, labels)

train_dataset = convert_to_tensors(tokenized_datasets['train'])
test_dataset = convert_to_tensors(tokenized_datasets['test'])

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)






class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.W1 = nn.Linear(hidden_size, hidden_size)
        self.W2 = nn.Linear(hidden_size, hidden_size)
        self.v = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        sequence_len = encoder_outputs.shape[1]
        hidden = hidden.unsqueeze(1).repeat(1, sequence_len, 1)

        energy = torch.tanh(self.W1(encoder_outputs) + self.W2(hidden))
        attention = self.v(energy).squeeze(2)
        attention_weights = torch.softmax(attention, dim=1)

        context = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs).squeeze(1)
        return context, attention_weights


class SimpleRecurrentNetworkWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, cell_type='GRU'):
        super(SimpleRecurrentNetworkWithAttention, self).__init__()

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.attention = Attention(hidden_size * 2)

        if cell_type == 'LSTM':
            self.rnn = nn.LSTM(hidden_size, hidden_size, batch_first=True, bidirectional=True)
        elif cell_type == 'GRU':
            self.rnn = nn.GRU(hidden_size, hidden_size, batch_first=True, bidirectional=True)
        else:
            self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True, bidirectional=True)

        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, inputs):
        embedded = self.embedding(inputs)
        rnn_output, hidden = self.rnn(embedded)

        if isinstance(hidden, tuple):
            hidden = hidden[0]

        hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)

        context, attention_weights = self.attention(hidden, rnn_output)

        output = self.fc(context)

        return output, attention_weights




vocab_size = tokenizer.vocab_size
hidden_size = 256
output_size = 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleRecurrentNetworkWithAttention(
    input_size=vocab_size,
    hidden_size=hidden_size,
    output_size=output_size,
    cell_type='GRU'
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

model.to(device)

def train(model, dataloader, criterion, optimizer, scheduler, device, print_every=100):
    model.train()
    running_loss = 0.0
    total_batches = len(dataloader)

    for batch_idx, (inputs, labels) in enumerate(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs, _ = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
        optimizer.step()

        running_loss += loss.item()

        if (batch_idx + 1) % 50 == 0:
            print(f"Batch [{batch_idx + 1}/{total_batches}], Loss: {loss.item():.4f}")

    scheduler.step()

    epoch_loss = running_loss / total_batches
    print(f"Epoch completed. Average Loss: {epoch_loss:.4f}")

    return epoch_loss

def decode_input(inputs, tokenizer):
    return tokenizer.decode(inputs, skip_special_tokens=True).split()

def evaluate(model, dataloader, criterion, device, tokenizer):
    model.eval()
    correct = 0
    total = 0
    total_loss = 0.0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs, attention_weights = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            if total == labels.size(0):
                print(f"Input sentence (token IDs): {inputs[0].cpu().numpy()}")

                words = decode_input(inputs[0], tokenizer)
                print(f"Decoded Input Sentence: {' '.join(words)}")

                attention_weights_np = attention_weights[0].cpu().numpy()

                print("Word attention distribution:")
                for word, attn in zip(words, attention_weights_np):
                    print(f"{word}: {attn:.4f}")

    accuracy = correct / total
    return total_loss / len(dataloader), accuracy


# 9. Training and evaluation
print(f"train loader size: {len(train_loader)}")
epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_loss = train(model, train_loader, criterion, optimizer, scheduler, device, print_every=100)

    # Evaluate the model and print attention visualization
    test_loss, test_accuracy = evaluate(model, test_loader, criterion, device, tokenizer)

    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
    print('-' * 50)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

train loader size: 1563
Epoch 1/5
Batch [50/1563], Loss: 0.6916
Batch [100/1563], Loss: 0.6875
Batch [150/1563], Loss: 0.6977
Batch [200/1563], Loss: 0.6790
Batch [250/1563], Loss: 0.6891
Batch [300/1563], Loss: 0.6894
Batch [350/1563], Loss: 0.6933
Batch [400/1563], Loss: 0.6876
Batch [450/1563], Loss: 0.6906
Batch [500/1563], Loss: 0.6999
Batch [550/1563], Loss: 0.6985
Batch [600/1563], Loss: 0.6909
Batch [650/1563], Loss: 0.6927
Batch [700/1563], Loss: 0.6960
Batch [750/1563], Loss: 0.6783
Batch [800/1563], Loss: 0.6941
Batch [850/1563], Loss: 0.6648
Batch [900/1563], Loss: 0.6798
Batch [950/1563], Loss: 0.6863
Batch [1000/1563], Loss: 0.6886
Batch [1050/1563], Loss: 0.6669
Batch [1100/1563], Loss: 0.6769
Batch [1150/1563], Loss: 0.6685
Batch [1200/1563], Loss: 0.6754
Batch [1250/1563], Loss: 0.6845
Batch [1300/1563], Loss: 0.6540
Batch [1350/1563], Loss: 0.6697
Batch [1400/1563], Loss: 0.6150
Batch [1450/1563], Loss: 0.6770
Batch [1500/1563], Loss: 0.6750
Batch [1550/1563], Loss: 0

In [ ]:
# Print the total number of parameters in the model
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")

# Separate trainable and non-trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
non_trainable_params = total_params - trainable_params

print(f"Trainable parameters: {trainable_params}")
print(f"Non-trainable parameters: {non_trainable_params}")

Total number of parameters: 65316866
Trainable parameters: 65316866
Non-trainable parameters: 0


In [ ]:
# Example list of sentences
sentences = [
    "i hate this movie",
    "this is the best film I've ever seen",
    "the plot was boring and slow",
    "i love the acting in this movie"
]

def classify(sentences, model, tokenizer, device):
    model.eval()  # Set model to evaluation mode

    # Iterate through each sentence in the list
    for sentence in sentences:
        print(f"\nProcessing sentence: '{sentence}'")

        # 1. Tokenize the input sentence
        inputs = tokenizer(sentence, return_tensors="pt", padding="max_length", truncation=True, max_length=512)

        # Move inputs to the correct device
        input_ids = inputs['input_ids'].to(device)

        # 2. Pass the tokenized sentence through the model to get predictions and attention weights
        with torch.no_grad():
            outputs, attention_weights = model(input_ids)

        # 3. Decode the input sentence to display the original words
        decoded_sentence = tokenizer.decode(input_ids[0], skip_special_tokens=True).split()

        # 4. Display the model's prediction
        _, predicted_label = torch.max(outputs, 1)
        predicted_label = predicted_label.item()
        sentiment = "Positive" if predicted_label == 1 else "Negative"
        print(f"Model Prediction: {sentiment}")

        # 5. Display the attention weights for each word in the input sentence
        attention_weights_np = attention_weights[0].cpu().numpy()  # Convert attention weights to NumPy array
        print("Word attention distribution:")
        for word, attn_weight in zip(decoded_sentence, attention_weights_np):
            print(f"{word}: {attn_weight:.4f}")

# Call the function with the list of sentences
classify(sentences, model, tokenizer, device)


Processing sentence: 'i hate this movie'
Model Prediction: Negative
Word attention distribution:
i: 0.0357
hate: 0.1564
this: 0.1558
movie: 0.2998

Processing sentence: 'this is the best film I've ever seen'
Model Prediction: Positive
Word attention distribution:
this: 0.0036
is: 0.0184
the: 0.0244
best: 0.0449
film: 0.7593
I've: 0.0665
ever: 0.0127
seen: 0.0350

Processing sentence: 'the plot was boring and slow'
Model Prediction: Negative
Word attention distribution:
the: 0.0206
plot: 0.0350
was: 0.0493
boring: 0.5536
and: 0.0517
slow: 0.0310

Processing sentence: 'i love the acting in this movie'
Model Prediction: Negative
Word attention distribution:
i: 0.0222
love: 0.0847
the: 0.0605
acting: 0.0475
in: 0.0718
this: 0.0489
movie: 0.0282
